# Praktikum Minggu 4: Pengumpulan Data (Data Collection)

**Mata Kuliah:** Big Data Analitik  
**Topik:** Web Scraping, REST API, dan Simulasi Data Streaming  
**Tujuan:** Mahasiswa mampu mengumpulkan data dari berbagai sumber menggunakan Python

---

*Week 4 Lab: Data Collection*  
*Topics covered: Web scraping with BeautifulSoup, REST API with Requests, streaming simulation, data storage*

In [ ]:
!pip install requests beautifulsoup4 lxml

## 1. Web Scraping dengan Requests & BeautifulSoup

Kita akan melakukan scraping dari **quotes.toscrape.com** — situs yang dirancang khusus untuk latihan web scraping. Situs ini berisi kutipan-kutipan beserta informasi penulis dan tag.

**Alur kerja web scraping:**
1. Kirim HTTP GET request ke URL target
2. Terima respons HTML
3. Parse HTML menggunakan BeautifulSoup
4. Ekstrak data yang diinginkan menggunakan CSS selector atau tag HTML
5. Simpan dalam format terstruktur

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import warnings
warnings.filterwarnings('ignore')

def scrape_quotes(base_url, max_pages=3):
    """Scrape kutipan dari quotes.toscrape.com."""
    semua_data = []
    headers = {"User-Agent": "Mozilla/5.0 BigDataCourse-Scraper/1.0"}

    for halaman in range(1, max_pages + 1):
        url = f"{base_url}/page/{halaman}/"
        try:
            respons = requests.get(url, headers=headers, timeout=10)
            respons.raise_for_status()
        except requests.RequestException as e:
            print(f"Gagal mengakses halaman {halaman}: {e}")
            break

        soup = BeautifulSoup(respons.text, "lxml")
        kutipan_list = soup.find_all("div", class_="quote")

        for kutipan in kutipan_list:
            teks = kutipan.find("span", class_="text").get_text(strip=True)
            penulis = kutipan.find("small", class_="author").get_text(strip=True)
            tags = [tag.get_text(strip=True)
                    for tag in kutipan.find_all("a", class_="tag")]
            semua_data.append({
                "teks": teks,
                "penulis": penulis,
                "tags": ", ".join(tags),
                "jumlah_tag": len(tags),
                "halaman": halaman
            })

        print(f"  Halaman {halaman}: {len(kutipan_list)} kutipan ditemukan")
        time.sleep(0.5)  # Etika: jeda antar permintaan

    return semua_data

print("Memulai scraping dari quotes.toscrape.com ...")
data_kutipan = scrape_quotes("http://quotes.toscrape.com", max_pages=3)

df_kutipan = pd.DataFrame(data_kutipan)
print(f"\nTotal kutipan berhasil di-scrape: {len(df_kutipan)}")
print("\n=== 5 Kutipan Pertama ===")
print(df_kutipan[["penulis", "teks", "jumlah_tag"]].head())

print("\n=== Penulis dengan Kutipan Terbanyak ===")
print(df_kutipan["penulis"].value_counts().head(5))

print("\n=== Statistik Jumlah Tag ===")
print(df_kutipan["jumlah_tag"].describe())

## 2. Parsing HTML Lokal / Simulasi

Kadang kita perlu mem-parsing HTML yang sudah tersimpan secara lokal atau yang dihasilkan oleh aplikasi kita sendiri. Contoh ini mensimulasikan parsing tabel HTML.

In [ ]:
# Contoh HTML lokal dengan tabel data produk
html_contoh = """
<!DOCTYPE html>
<html>
<head><title>Daftar Produk Toko Online</title></head>
<body>
  <h1>Katalog Produk</h1>
  <div id="info">
    <p class="update">Terakhir diperbarui: 2024-01-15</p>
    <p class="total">Total produk: 5 item</p>
  </div>
  <table id="tabel-produk">
    <thead>
      <tr>
        <th>ID</th><th>Nama Produk</th><th>Kategori</th>
        <th>Harga (Rp)</th><th>Stok</th><th>Rating</th>
      </tr>
    </thead>
    <tbody>
      <tr><td>P001</td><td>Laptop UltraBook Pro</td><td>Elektronik</td><td>12500000</td><td>15</td><td>4.7</td></tr>
      <tr><td>P002</td><td>Smartphone X200</td><td>Elektronik</td><td>3800000</td><td>42</td><td>4.5</td></tr>
      <tr><td>P003</td><td>Meja Belajar Ergonomis</td><td>Furnitur</td><td>850000</td><td>8</td><td>4.3</td></tr>
      <tr><td>P004</td><td>Headphone Noise Cancelling</td><td>Elektronik</td><td>1200000</td><td>25</td><td>4.6</td></tr>
      <tr><td>P005</td><td>Buku Python Data Science</td><td>Buku</td><td>185000</td><td>100</td><td>4.8</td></tr>
    </tbody>
  </table>
  <ul class="promo-list">
    <li data-promo="DISKON10">Diskon 10% untuk pembelian pertama</li>
    <li data-promo="GRATIS_ONGKIR">Gratis ongkir untuk pembelian di atas Rp 200.000</li>
  </ul>
</body>
</html>
"""

soup = BeautifulSoup(html_contoh, "lxml")

# Ekstrak informasi umum
print("=== Informasi Halaman ===")
print(f"Judul  : {soup.title.get_text()}")
print(f"Update : {soup.find('p', class_='update').get_text()}")
print(f"Total  : {soup.find('p', class_='total').get_text()}")

# Ekstrak tabel produk
print("\n=== Ekstraksi Tabel Produk ===")
tabel = soup.find("table", id="tabel-produk")
header = [th.get_text(strip=True) for th in tabel.find_all("th")]
baris_data = []
for tr in tabel.find("tbody").find_all("tr"):
    nilai = [td.get_text(strip=True) for td in tr.find_all("td")]
    baris_data.append(dict(zip(header, nilai)))

df_produk = pd.DataFrame(baris_data)
df_produk["Harga (Rp)"] = df_produk["Harga (Rp)"].astype(int)
df_produk["Stok"] = df_produk["Stok"].astype(int)
df_produk["Rating"] = df_produk["Rating"].astype(float)
print(df_produk.to_string(index=False))

# Ekstrak daftar promo
print("\n=== Kode Promo ===")
for li in soup.find_all("li", attrs={"data-promo": True}):
    print(f"  [{li['data-promo']}] {li.get_text(strip=True)}")

# Analisis sederhana
print("\n=== Analisis Produk ===")
print(f"Rata-rata harga  : Rp {df_produk['Harga (Rp)'].mean():,.0f}")
print(f"Produk termahal  : {df_produk.loc[df_produk['Harga (Rp)'].idxmax(), 'Nama Produk']}")
print(f"Rating tertinggi : {df_produk.loc[df_produk['Rating'].idxmax(), 'Nama Produk']}")

## 3. Mengakses REST API Publik

Kita akan menggunakan **JSONPlaceholder** (https://jsonplaceholder.typicode.com) — API palsu gratis untuk testing dan prototyping. API ini menyediakan data posts, comments, users, todos, dll.

In [ ]:
import json

BASE_URL = "https://jsonplaceholder.typicode.com"
headers = {"Content-Type": "application/json"}

# --- GET: Ambil semua posts ---
print("=== GET /posts ===")
resp_posts = requests.get(f"{BASE_URL}/posts", headers=headers, timeout=10)
print(f"Status Code : {resp_posts.status_code}")
print(f"Content-Type: {resp_posts.headers.get('content-type', 'N/A')}")

posts = resp_posts.json()
df_posts = pd.DataFrame(posts)
print(f"Jumlah posts: {len(df_posts)}")
print(df_posts.head(3).to_string())

# --- GET: Ambil satu post berdasarkan ID ---
print("\n=== GET /posts/1 ===")
resp_satu = requests.get(f"{BASE_URL}/posts/1", timeout=10)
print(json.dumps(resp_satu.json(), indent=2))

# --- GET: Ambil comments untuk post tertentu ---
print("\n=== GET /posts/1/comments ===")
resp_comments = requests.get(f"{BASE_URL}/posts/1/comments", timeout=10)
df_comments = pd.DataFrame(resp_comments.json())
print(f"Jumlah komentar: {len(df_comments)}")
print(df_comments[["id", "name", "email"]].to_string(index=False))

# --- GET: Ambil data users ---
print("\n=== GET /users ===")
resp_users = requests.get(f"{BASE_URL}/users", timeout=10)
df_users = pd.DataFrame(resp_users.json())
print(f"Jumlah users: {len(df_users)}")
print(df_users[["id", "name", "username", "email"]].to_string(index=False))

# --- Analisis: Jumlah post per user ---
print("\n=== Analisis: Jumlah Post per User ===")
posts_per_user = df_posts.groupby("userId")["id"].count().reset_index()
posts_per_user.columns = ["userId", "jumlah_post"]
print(posts_per_user.to_string(index=False))

# --- Analisis: Panjang rata-rata judul post per user ---
df_posts["panjang_judul"] = df_posts["title"].str.len()
print("\n=== Rata-rata Panjang Judul per User (5 user pertama) ===")
print(df_posts.groupby("userId")["panjang_judul"].mean().head().round(1))

## 4. Simulasi Data Streaming

Kita akan mensimulasikan data streaming menggunakan **generator function** Python yang menghasilkan rekaman data secara bertahap, seperti sensor IoT atau log server yang terus menghasilkan data.

In [ ]:
import random
from datetime import datetime, timedelta
import time as time_module

# --- Generator: Simulasi sensor IoT ---
def generator_sensor_iot(n_records=20, device_count=5):
    """Generator yang mensimulasikan data dari beberapa sensor IoT."""
    random.seed(42)
    waktu_mulai = datetime.now()
    device_ids = [f"SENSOR_{i:03d}" for i in range(1, device_count + 1)]
    lokasi_map = {
        "SENSOR_001": "Ruang Server",
        "SENSOR_002": "Lab Komputer A",
        "SENSOR_003": "Lab Komputer B",
        "SENSOR_004": "Ruang Kuliah",
        "SENSOR_005": "Koridor",
    }

    for i in range(n_records):
        device = random.choice(device_ids)
        record = {
            "record_id": i + 1,
            "timestamp": (waktu_mulai + timedelta(seconds=i * 3)).strftime("%Y-%m-%d %H:%M:%S"),
            "device_id": device,
            "lokasi": lokasi_map[device],
            "suhu_celsius": round(random.uniform(18.0, 35.0), 1),
            "kelembaban_persen": round(random.uniform(40.0, 90.0), 1),
            "status": random.choices(["normal", "warning", "critical"], weights=[0.7, 0.2, 0.1])[0]
        }
        yield record

# --- Kumpulkan data dari generator ---
print("Mengumpulkan data dari stream sensor IoT ...")
stream_buffer = []
for i, record in enumerate(generator_sensor_iot(n_records=20)):
    stream_buffer.append(record)
    if (i + 1) % 5 == 0:
        print(f"  [{record['timestamp']}] Diterima {i + 1} record dari stream")

df_stream = pd.DataFrame(stream_buffer)

print(f"\nTotal record terkumpul: {len(df_stream)}")
print("\n=== Sampel Data Streaming ===")
print(df_stream.to_string(index=False))

# --- Analisis data streaming ---
print("\n=== Analisis Real-time ===")
print("\nRata-rata suhu per lokasi:")
print(df_stream.groupby("lokasi")["suhu_celsius"].mean().round(2).to_string())

print("\nDistribusi status sensor:")
print(df_stream["status"].value_counts().to_string())

print("\nRecord dengan status CRITICAL:")
df_critical = df_stream[df_stream["status"] == "critical"]
if len(df_critical) > 0:
    print(df_critical[["timestamp", "device_id", "lokasi", "suhu_celsius"]].to_string(index=False))
else:
    print("  Tidak ada record critical")

print(f"\nSuhu tertinggi  : {df_stream['suhu_celsius'].max()}°C ({df_stream.loc[df_stream['suhu_celsius'].idxmax(), 'lokasi']})")
print(f"Kelembaban rata-rata: {df_stream['kelembaban_persen'].mean():.1f}%")

## 5. Menyimpan Data yang Dikumpulkan

Setelah mengumpulkan data, langkah berikutnya adalah menyimpannya dalam format yang sesuai untuk analisis lebih lanjut. Format umum: **CSV** (tabular), **JSON** (semi-structured), **Parquet** (columnar, efisien).

In [ ]:
import os

os.makedirs("output_data", exist_ok=True)

# --- Simpan ke CSV ---
path_csv = "output_data/data_sensor.csv"
df_stream.to_csv(path_csv, index=False, encoding="utf-8")
print(f"Disimpan ke CSV: {path_csv} ({os.path.getsize(path_csv)} bytes)")

# --- Simpan ke JSON ---
path_json = "output_data/data_sensor.json"
df_stream.to_json(path_json, orient="records", indent=2, force_ascii=False)
print(f"Disimpan ke JSON: {path_json} ({os.path.getsize(path_json)} bytes)")

# --- Simpan kutipan ke CSV jika tersedia ---
if 'df_kutipan' in dir() and len(df_kutipan) > 0:
    path_kutipan = "output_data/kutipan_scraped.csv"
    df_kutipan.to_csv(path_kutipan, index=False, encoding="utf-8")
    print(f"Disimpan ke CSV: {path_kutipan} ({os.path.getsize(path_kutipan)} bytes)")

# --- Baca ulang dan verifikasi CSV ---
print("\n=== Verifikasi — Baca Ulang CSV ===")
df_verif_csv = pd.read_csv(path_csv)
print(f"Shape: {df_verif_csv.shape}")
print(f"Kolom: {list(df_verif_csv.columns)}")
print(df_verif_csv.head(3).to_string(index=False))

# --- Baca ulang dan verifikasi JSON ---
print("\n=== Verifikasi — Baca Ulang JSON ===")
df_verif_json = pd.read_json(path_json)
print(f"Shape: {df_verif_json.shape}")
print(f"Kolom: {list(df_verif_json.columns)}")
print(df_verif_json.head(3).to_string(index=False))

# --- Perbandingan integritas data ---
print("\n=== Integritas Data ===")
assert len(df_verif_csv) == len(df_stream), "Jumlah baris CSV tidak sesuai!"
assert len(df_verif_json) == len(df_stream), "Jumlah baris JSON tidak sesuai!"
print(f"CSV  : {len(df_verif_csv)} baris — OK")
print(f"JSON : {len(df_verif_json)} baris — OK")
print("Semua data tersimpan dengan integritas penuh!")

## Tugas Praktikum

Kerjakan soal-soal berikut secara mandiri:

**Soal 1 — Web Scraping Multi-halaman:**  
Lakukan scraping dari `http://quotes.toscrape.com` untuk **semua halaman** yang tersedia (hingga tidak ada lagi tombol "Next"). Analisis: siapa penulis dengan kutipan terbanyak? Tag apa yang paling sering muncul?

**Soal 2 — REST API Lanjutan:**  
Gunakan JSONPlaceholder API untuk:
- Ambil data `/todos` dan hitung persentase tugas yang sudah selesai (`completed=True`) per user
- Buat visualisasi bar chart dari hasil tersebut menggunakan matplotlib

**Soal 3 — Scraping dengan BeautifulSoup:**  
Buat HTML string baru berisi tabel jadwal kuliah dengan kolom: `hari`, `jam`, `mata_kuliah`, `dosen`, `ruangan`. Isi dengan minimal 10 baris. Ekstrak datanya menggunakan BeautifulSoup dan simpan ke CSV.

**Soal 4 — Simulasi Streaming Lanjutan:**  
Modifikasi generator sensor IoT untuk:
- Menghasilkan 100 rekaman
- Mendeteksi secara real-time jika ada suhu > 30°C dan kelembaban > 80% secara bersamaan (kondisi kritis)
- Hitung rata-rata suhu bergerak (rolling average) setiap 10 rekaman

In [ ]:
# Ringkasan hasil praktikum
print("=" * 55)
print("   RINGKASAN PRAKTIKUM MINGGU 4 — DATA COLLECTION")
print("=" * 55)

hasil_summary = {
    "Web Scraping": f"{len(df_kutipan) if 'df_kutipan' in dir() else 0} kutipan berhasil di-scrape",
    "Parsing HTML Lokal": f"{len(df_produk)} produk berhasil diekstrak",
    "REST API (posts)": f"{len(df_posts)} posts diambil dari JSONPlaceholder",
    "REST API (users)": f"{len(df_users)} users diambil dari JSONPlaceholder",
    "Data Streaming": f"{len(df_stream)} rekaman sensor IoT dikumpulkan",
    "File CSV": f"Disimpan di {path_csv}",
    "File JSON": f"Disimpan di {path_json}",
}

for topik, hasil in hasil_summary.items():
    print(f"  ✓ {topik:<25}: {hasil}")

print("=" * 55)
print("Praktikum Minggu 4 selesai!")